In [20]:
%matplotlib inline

In [21]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from tempfile import NamedTemporaryFile
from IPython.display import HTML
import scipy.integrate as integrate
from scipy.spatial.distance import pdist, squareform
from scipy.fftpack import fft, ifft

In [22]:
VIDEO_TAG = """<video controls>
     <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
     Your browser does not support the video tag.
    </video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")

    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

# This part is of projectile

In [40]:
def projectile(u, theta, n=1000, g=9.81):
    '''The motion of a point mass under gravity in two dimensions thrown at an ang\
    le theta with the horizontal and a velocity'''
    theta_rad = theta*np.pi/180.0
    t_max = 2*u*np.sin(abs(theta_rad))/9.81
    t = np.linspace(0.0, t_max, n)
    x = u*np.cos(theta_rad)*t
    y = u*np.sin(theta_rad)*t - 0.5*g*t*t
    return t, x, y


def animate(X,Y):
    fig = plt.figure()
    ax = plt.axes(xlim=(np.amin(X), np.amax(X)), ylim=(np.amin(Y)-0.1, np.amax(Y)+0.1))
    line, = ax.plot([], [], lw=2)

    def init():
        line.set_data([],[])
        return line,
    def animate(i):
        x = X[:i]
        y = Y[:i]
        line.set_data(x, y)
        return line,

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=1000, interval=5, blit=True)
    return anim

In [41]:
t, x, y = projectile(10.0, 45.0)

In [42]:
display_animation(animate(x,y))

# This part is of Cyclotron

In [43]:
def euler(dt, tf, q=5.0, m=10.0, E=np.array([0.0, 0.0, 0.0]), Bi=np.array([0.0, 0.0, 1.0]), gradBx=0.0, gradBy=0.0, ui=np.array([10**4, 0.0, 0.0]), x0=np.array([0.0, 0.0, 0.0])):
    ntime = int(tf/dt)
    v_euler = np.zeros((ntime+1, 3))
    x_euler = np.zeros((ntime + 1, 3))
    x_euler[0] = x0
    v_euler[0] = ui
    B = Bi.copy()
    T = np.linspace(0.0, tf, ntime+1)
    for i in range(1, ntime + 1):
        B[2] = Bi[2] + (x_euler[i-1, 0] - x0[0])*gradBx + (x_euler[i-1, 1] - x0[1])*gradBy
        v_euler[i] = v_euler[i-1] + q*(E + np.cross(v_euler[i-1], B))/m*dt
        x_euler[i] = x_euler[i-1] + v_euler[i-1]*dt
    e_euler = 0.5*m*(v_euler[:, 0]**2 + v_euler[:, 1]**2 + v_euler[:, 2]**2)
    return x_euler, e_euler, T

def euler2(dt, tf, q=5.0, m=10.0, E=np.array([0.0, 0.0, 0.0]), Bi=np.array([0.0, 0.0, 1.0]), gradBx=0.0, gradBy=0.0, ui=np.array([10**4, 0.0, 0.0]), x0=np.array([0.0, 0.0, 0.0])):
    ntime = int(tf/dt)
    v_euler2 = np.zeros((ntime+1, 3))
    x_euler2 = np.zeros((ntime + 1, 3))
    x_euler2[0] = x0
    v_euler2[0] = ui
    T = np.linspace(0.0, tf, ntime+1)
    B = Bi.copy()
    for i in range(1, ntime + 1):
        B[2] = Bi[2] + (x_euler2[i-1, 0] - x0[0])*gradBx + (x_euler2[i-1, 1] - x0[1])*gradBy
        v_euler2[i] = v_euler2[i-1]+q*(E + np.cross(v_euler2[i-1], B))/m*dt
        x_euler2[i] = x_euler2[i-1] + v_euler2[i]*dt
    e_euler2 = 0.5*m*(v_euler2[:, 0]**2+v_euler2[:, 1]**2+v_euler2[:, 2]**2)
    return x_euler2, e_euler2, T


def RK2(dt, tf, q=5.0, m=10.0, E=np.array([0.0, 0.0, 0.0]), Bi=np.array([0.0, 0.0, 1.0]), gradBx=0.0, gradBy=0.0, ui=np.array([10**4, 0.0, 0.0]), x0=np.array([0.0, 0.0, 0.0])):
    ntime = int(tf/dt)
    v_RK2 = np.zeros((ntime+1, 3))
    x_RK2 = np.zeros((ntime + 1, 3))
    x_RK2[0] = x0
    v_RK2[0] = ui
    T = np.linspace(0.0, tf, ntime+1)
    B = Bi.copy()
    for i in range(1, ntime + 1):
        B[2] = Bi[2] + (x_RK2[i-1, 0] - x0[0])*gradBx + (x_RK2[i-1, 1]-x0[1])*gradBy
        kv1 = q*(E + np.cross(v_RK2[i-1], B))/m*dt
        kx1 = v_RK2[i-1]*dt
        kv2 = q*(E + np.cross(v_RK2[i-1]+kv1/2.0, B))/m*dt
        kx2 = (v_RK2[i-1] + kv1)*dt
        v_RK2[i] = v_RK2[i-1] + kv2
        x_RK2[i] = x_RK2[i-1] + kx2
    e_RK2 = 0.5*m*(v_RK2[:, 0]**2 + v_RK2[:, 1]**2 + v_RK2[:, 2]**2)
    return x_RK2, e_RK2, T


def boris(dt, tf, q=5.0, m=10.0, E=np.array([0.0, 0.0, 0.0]), Bi=np.array([0.0, 0.0, 1.0]), gradBx=0.0, gradBy=0.0, ui=np.array([10**4, 0.0, 0.0]), x0=np.array([0.0, 0.0, 0.0])):
    ntime = int(tf/dt)
    v_boris = np.zeros((ntime+1, 3))
    x_boris = np.zeros((ntime + 1, 3))
    x_boris[0] = x0
    v_boris[0] = ui
    T = np.linspace(0.0, tf, ntime+1)
    B = Bi.copy()
    for i in range(1, ntime+1):
        vplus = np.zeros(3)
        B[2] = Bi[2] + (x_boris[i-1, 0] - x0[0])*gradBx + (x_boris[i-1, 1] - x0[1])*gradBy
        alpha = 0.5*q*B[2]*dt/m
        vminus = v_boris[i-1] + 0.5*q*E/m
        vplus[0] = vminus[0]*(1-alpha**2)/(1+alpha**2) + 2*alpha/(1+alpha**2)*vminus[1]
        vplus[1] = vminus[1]*(1-alpha**2)/(1+alpha**2) - 2*alpha/(1+alpha**2)*vminus[0]
        v_boris[i] = vplus + 0.5*q*E/m
        x_boris[i] = x_boris[i-1] + v_boris[i]*dt
    e_boris = 0.5*m*(v_boris[:, 0]**2 + v_boris[:, 1]**2 + v_boris[:, 2]**2)
    return x_boris, e_boris, T

In [44]:
x_boris, e_boris, T = boris(0.1, 100, E=np.array([10**5, 0.0, 0.0]), gradBx=0.1, gradBy=0.1)

In [46]:
display_animation(animate(x_boris[:, 0], x_boris[:, 1]))

# This part is for particles in a box.

In [48]:
class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, -1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[-2, 2, -2, 2],
                 size=0.04,
                 M=0.05,
                 G=9.8):
        self.init_state = np.asarray(init_state, dtype=float)
        self.M = M * np.ones(self.init_state.shape[0])
        self.size = size
        self.state = self.init_state.copy()
        self.time_elapsed = 0
        self.bounds = bounds
        self.G = G

    def step(self, dt):
        self.time_elapsed += dt
        self.state[:, :2] += dt * self.state[:, 2:]
        D = squareform(pdist(self.state[:, :2]))
        ind1, ind2 = np.where(D < 2 * self.size)
        unique = (ind1 < ind2)
        ind1 = ind1[unique]
        ind2 = ind2[unique]
        for i1, i2 in zip(ind1, ind2):
            m1 = self.M[i1]
            m2 = self.M[i2]
            r1 = self.state[i1, :2]
            r2 = self.state[i2, :2]
            v1 = self.state[i1, 2:]
            v2 = self.state[i2, 2:]
            r_rel = r1 - r2
            v_rel = v1 - v2
            v_cm = (m1 * v1 + m2 * v2) / (m1 + m2)
            rr_rel = np.dot(r_rel, r_rel)
            vr_rel = np.dot(v_rel, r_rel)
            v_rel = 2 * r_rel * vr_rel / rr_rel - v_rel
            self.state[i1, 2:] = v_cm + v_rel * m2 / (m1 + m2)
            self.state[i2, 2:] = v_cm - v_rel * m1 / (m1 + m2)
        crossed_x1 = (self.state[:, 0] < self.bounds[0] + self.size)
        crossed_x2 = (self.state[:, 0] > self.bounds[1] - self.size)
        crossed_y1 = (self.state[:, 1] < self.bounds[2] + self.size)
        crossed_y2 = (self.state[:, 1] > self.bounds[3] - self.size)
        self.state[crossed_x1, 0] = self.bounds[0] + self.size
        self.state[crossed_x2, 0] = self.bounds[1] - self.size
        self.state[crossed_y1, 1] = self.bounds[2] + self.size
        self.state[crossed_y2, 1] = self.bounds[3] - self.size
        self.state[crossed_x1 | crossed_x2, 2] *= -1
        self.state[crossed_y1 | crossed_y2, 3] *= -1
        self.state[:, 3] -= self.M * self.G * dt

np.random.seed(0)


def Particles(dt=1./30, init_state=-0.5 + np.random.random((50, 4))):
    init_state[:, :2] *= 3.9
    box = ParticleBox(init_state, size=0.04)
    return init_state, box


def animate(name, dt=1./30, init_state=-0.5 + np.random.random((50, 4))):
    init_state, box = Particles(dt, init_state)
    fig = plt.figure()
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=(-3.2, 3.2), ylim=(-2.4, 2.4))

    particles, = ax.plot([], [], 'bo', ms=6)

    rect = plt.Rectangle(box.bounds[::2],
                         box.bounds[1] - box.bounds[0],
                         box.bounds[3] - box.bounds[2],
                         ec='none', lw=2, fc='none')
    ax.add_patch(rect)

    def init():
        # global box, rect
        particles.set_data([], [])
        rect.set_edgecolor('none')
        return particles, rect

    def animate(i):
        # global box, rect, dt, ax, fig
        box.step(dt)

        ms = int(fig.dpi * 2 * box.size * fig.get_figwidth() /
                 np.diff(ax.get_xbound())[0])
        rect.set_edgecolor('k')
        particles.set_data(box.state[:, 0], box.state[:, 1])
        particles.set_markersize(ms)
        return particles, rect

    ani = animation.FuncAnimation(fig, animate, frames=600,
                                  interval=10, blit=True, init_func=init)
    # ani.save(name+'.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

    return ani

In [49]:
display_animation(animate('particle_box'))

# This is for Schrodinger's Barrier Problem

In [50]:
class Schrodinger(object):
    def __init__(self, x, psi_x0, V_x,
                 k0=None, hbar=1, m=1, t0=0.0):
        self.x, psi_x0, self.V_x = map(np.asarray, (x, psi_x0, V_x))
        N = self.x.size
        assert self.x.shape == (N,)
        assert psi_x0.shape == (N,)
        assert self.V_x.shape == (N,)
        self.hbar = hbar
        self.m = m
        self.t = t0
        self.dt_ = None
        self.N = len(x)
        self.dx = self.x[1] - self.x[0]
        self.dk = 2 * np.pi / (self.N * self.dx)
        if k0 is None:
            self.k0 = -0.5 * self.N * self.dk
        else:
            self.k0 = k0
        self.k = self.k0 + self.dk * np.arange(self.N)
        self.psi_x = psi_x0
        self.compute_k_from_x()
        self.x_evolve_half = None
        self.x_evolve = None
        self.k_evolve = None
        self.psi_x_line = None
        self.psi_k_line = None
        self.V_x_line = None

    def _set_psi_x(self, psi_x):
        self.psi_mod_x = (psi_x * np.exp(-1j * self.k[0] * self.x) *
                          self.dx / np.sqrt(2 * np.pi))

    def _get_psi_x(self):
        return (self.psi_mod_x * np.exp(1j * self.k[0] * self.x) *
                np.sqrt(2 * np.pi) / self.dx)

    def _set_psi_k(self, psi_k):
        self.psi_mod_k = psi_k * np.exp(1j * self.x[0] *
                                        self.dk * np.arange(self.N))

    def _get_psi_k(self):
        return self.psi_mod_k * np.exp(-1j * self.x[0] *
                                       self.dk * np.arange(self.N))

    def _get_dt(self):
        return self.dt_

    def _set_dt(self, dt):
        if dt != self.dt_:
            self.dt_ = dt
            self.x_evolve_half = np.exp(-0.5 * 1j * self.V_x /
                                        self.hbar * dt)
            self.x_evolve = self.x_evolve_half * self.x_evolve_half
            self.k_evolve = np.exp(-0.5 * 1j * self.hbar /
                                   self.m * (self.k * self.k) * dt)
    psi_x = property(_get_psi_x, _set_psi_x)
    psi_k = property(_get_psi_k, _set_psi_k)
    dt = property(_get_dt, _set_dt)

    def compute_k_from_x(self):
        self.psi_mod_k = fft(self.psi_mod_x)

    def compute_x_from_k(self):
        self.psi_mod_x = ifft(self.psi_mod_k)

    def time_step(self, dt, Nsteps=1):
        self.dt = dt
        if Nsteps > 0:
            self.psi_mod_x *= self.x_evolve_half
        for i in xrange(Nsteps - 1):
            self.compute_k_from_x()
            self.psi_mod_k *= self.k_evolve
            self.compute_x_from_k()
            self.psi_mod_x *= self.x_evolve
        self.compute_k_from_x()
        self.psi_mod_k *= self.k_evolve
        self.compute_x_from_k()
        self.psi_mod_x *= self.x_evolve_half
        self.compute_k_from_x()
        self.t += dt * Nsteps


def gauss_x(x, a, x0, k0):
    return ((a * np.sqrt(np.pi)) ** (-0.5) *
            np.exp(-0.5 * ((x - x0) * 1. / a) ** 2 + 1j * x * k0))


def gauss_k(k, a, x0, k0):
    return ((a / np.sqrt(np.pi))**0.5 *
            np.exp(-0.5 * (a * (k - k0)) ** 2 - 1j * (k - k0) * x0))


def theta(x):
    x = np.asarray(x)
    y = np.zeros(x.shape)
    y[x > 0] = 1.0
    return y


def square_barrier(x, width, height):
    return height * (theta(x) - theta(x - width))


hbar = 1.0


def animate(name, dt=0.01, N_steps=50, t_max=120, m=1.9, N=2 ** 11, dx=0.1, V0=1.5):
    frames = int(t_max / float(N_steps * dt))
    x = dx * (np.arange(N) - 0.5 * N)
    L = hbar / np.sqrt(2 * m * V0)
    a = 3 * L
    x0 = -60 * L
    V_x = square_barrier(x, a, V0)
    V_x[x < -98] = 1E6
    V_x[x > 98] = 1E6
    p0 = np.sqrt(2 * m * 0.2 * V0)
    dp2 = p0 * p0 * 1./80
    d = hbar / np.sqrt(2 * dp2)

    k0 = p0 / hbar
    v0 = p0 / m
    psi_x0 = gauss_x(x, d, x0, k0)
    S = Schrodinger(x=x,
                    psi_x0=psi_x0,
                    V_x=V_x,
                    hbar=hbar,
                    m=m,
                    k0=-28)
    fig = plt.figure()
    xlim = (-100, 100)
    klim = (-5, 5)
    ymin = 0
    ymax = V0
    ax1 = fig.add_subplot(211, xlim=xlim,
                          ylim=(ymin - 0.2 * (ymax - ymin),
                                ymax + 0.2 * (ymax - ymin)))
    psi_x_line, = ax1.plot([], [], c='r', label=r'$|\psi(x)|$')
    V_x_line, = ax1.plot([], [], c='k', label=r'$V(x)$')
    center_line = ax1.axvline(0, c='k', ls=':',
                              label=r"$x_0 + v_0t$")
    title = ax1.set_title("")
    ax1.legend(prop=dict(size=12))
    ax1.set_xlabel('$x$')
    ax1.set_ylabel(r'$|\psi(x)|$')
    ymin = abs(S.psi_k).min()
    ymax = abs(S.psi_k).max()
    ax2 = fig.add_subplot(212, xlim=klim,
                          ylim=(ymin - 0.2 * (ymax - ymin),
                                ymax + 0.2 * (ymax - ymin)))
    psi_k_line, = ax2.plot([], [], c='r', label=r'$|\psi(k)|$')
    p0_line1 = ax2.axvline(-p0 / hbar, c='k', ls=':', label=r'$\pm p_0$')
    p0_line2 = ax2.axvline(p0 / hbar, c='k', ls=':')
    mV_line = ax2.axvline(np.sqrt(2 * V0) / hbar, c='k', ls='--',
                          label=r'$\sqrt{2mV_0}$')
    ax2.legend(prop=dict(size=12))
    ax2.set_xlabel('$k$')
    ax2.set_ylabel(r'$|\psi(k)|$')
    V_x_line.set_data(S.x, S.V_x)

    def init():
        psi_x_line.set_data([], [])
        V_x_line.set_data([], [])
        center_line.set_data([], [])
        psi_k_line.set_data([], [])
        title.set_text("")
        return (psi_x_line, V_x_line, center_line, psi_k_line, title)

    def animate(i):
        S.time_step(dt, N_steps)
        psi_x_line.set_data(S.x, 4 * abs(S.psi_x))
        V_x_line.set_data(S.x, S.V_x)
        center_line.set_data(2 * [x0 + S.t * p0 / m], [0, 1])
        psi_k_line.set_data(S.k, abs(S.psi_k))
        title.set_text("t = %.2f" % S.t)
        return (psi_x_line, V_x_line, center_line, psi_k_line, title)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=frames, interval=30, blit=True)

    # anim.save('schrodinger.mp4', fps=15, extra_args=['-vcodec', 'libx264'])

    return anim


In [ ]:
display_animation(animate('scrodinger'))